# Разведочный анализ данных (EDA)

In [1]:
import pandas as pd
import json
import ast
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy
import scipy.stats as stats
import sklearn.linear_model as lm
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.preprocessing import MultiLabelBinarizer

%matplotlib inline
# !gdown --id 12ZK2_dILTS_22sNFtIOT7MjaojKPOQoo
# !gdown --id 1TeAXhAVS7TjFVpibo1Td_i-Xo5ZW3kt2

pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.options.display.max_columns = 20
pd.options.display.max_rows = 100
pd.options.display.min_rows = 100

In [2]:
df = pd.read_excel('region roles (26.05.22).xlsx')

In [3]:
# Удаляем неинформативные столбцы.

df.drop('immediate_redirect_url', axis=1, inplace=True)
df.drop('id', axis=1, inplace=True)
df.drop('premium', axis=1, inplace=True)
df.drop('Unnamed: 0', axis=1, inplace=True)
df.drop('response_url', axis=1, inplace=True)
df.drop('response_letter_required', axis=1, inplace=True)
df.drop('has_test', axis=1, inplace=True)
df.drop('address', axis=1, inplace=True)
df.drop('sort_point_distance', axis=1, inplace=True)
df.drop('url', axis=1, inplace=True)
df.drop('insider_interview', axis=1, inplace=True)
df.drop('alternate_url', axis=1, inplace=True)
df.drop('working_days', axis=1, inplace=True)
df.drop('working_time_intervals', axis=1, inplace=True)
df.drop('working_time_modes', axis=1, inplace=True)
df.drop('accept_temporary', axis=1, inplace=True)
df.drop('contacts', axis=1, inplace=True)
df.drop('relations', axis=1, inplace=True)
# df.drop('schedule', axis=1, inplace=True)
df.drop('published_at', axis=1, inplace=True)
df.drop('created_at', axis=1, inplace=True)
df.drop('type', axis=1, inplace=True)

df.drop(df[df['archived'] == True].index, axis=0, inplace=True)
df.drop('archived', axis=1, inplace=True)



In [4]:
# Преобразуем ячейки сета в удобочитаемый вид.
course = {'RUR': 1, 'USD': 59.74, 'EUR': 59.53, 'KZT': 0.1254} # курс валют на 23.08.2022

def specializations(data):
    result = []
    for spec in eval(data):
        result.append((spec["name"],spec["profarea_name"]))
    return tuple(result)

def salary_eval(data, key):
    if data is np.nan:
        return np.nan
    try:
        salary = ast.literal_eval(data)
    except:
        return np.nan
    if not isinstance(salary, dict):
        return np.nan
    if salary.get(key) is None:
        return np.nan
    return salary[key] * course[salary['currency']]
# df['area'] = df['area'].apply(lambda x : ast.literal_eval(x)['name'])
df['key_skills'] = df['key_skills'].apply(lambda x : tuple([name['name'] for name in eval(x)]))
df['experience'] = df['experience'].apply(lambda x : ast.literal_eval(x)['name'])
df['department'] = df['department'].apply(lambda x : ast.literal_eval(x)['name'] if not(x is np.nan) else np.nan)
df['employer'] = df['employer'].apply(lambda x : ast.literal_eval(x)['name'])
df["schedule"] = df["schedule"].apply(lambda x : ast.literal_eval(x)['name'])
df['specializations'] = df['specializations'].apply(specializations)

df['salary_min'] = df.salary.apply(salary_eval, key='from')
df['salary_max'] = df.salary.apply(salary_eval, key='to')
df['salary_mean'] = df[['salary_min','salary_max']].mean(axis=1)

In [5]:
# Приводим основные профессиональные навыки к одинаковому написанию.
skills_dict = {'sql' : 'SQL', 'git': 'Git', 'linux' : 'linux', 'python' : 'Python', 'java script' : 'Java Script',
               'javascript' : 'Java Script' ,'json' : 'JSON', 'js' : 'Java Script', 'xml' : 'XML', '1c' : '1C',
               '1 c' : '1C', '1с' : '1C', '1 с' : '1C', 'html' : 'HTML', 'css' : 'CSS', 'c#' : 'C#', 'с#' : 'C#',
               'php' : 'PHP', 'docker' : 'Docker', 'rest' : 'REST', 'nix' : 'Nix', 'ооп' : 'ООП', 'jira' : 'Atlassian Jira ',
               'с++' : 'C++', 'c++' : 'C++', 'android' : 'Android', 'swift' : 'Swift/iOS', 'ios' : 'Swift/iOS',
               'kotlin' : 'Kotlin', 'tcp' : 'TCP/IP', 'информационные технологии' : 'Информационные технологии',
               'информационная безопасность': 'Информационная безопасность', 'информационной безопасности': 'Информационная безопасность', 
               'dos': 'DOS', 'net' : '.NET',
               'api' : 'API','active directory' : 'Active directory','jquery' : 'JQuery',
               'настройка пк' : 'Настройка ПК','настройка сетевых подключений' : 'Настройка сетевых подключений',
               'сетевые технологии': 'Настройка сетевых подключений',
               'настройка по' : 'Настройка ПО', 'установка по' : 'Настройка ПО',
               'windows' : 'Windows', 'базам' : 'Работа с БД', 'базы дан' : 'Работа с БД',
               'субд' : 'Работа с БД',
               'техническая поддержка' : 'Техническая поддержка','сетевог' : 'Настройка сетевого оборудования','crm' : 'Crm',
               'react' : 'React','typescript' : 'TypeScript','dhcp' : 'DHCP',
               'анализ' : 'Анализ данных','bash' : 'Bash','zabbix' : 'Zabbix','laravel' : 'Laravel',
               'dns': 'Dns', 'cisco': 'Cisco', 'bpmn': 'BPMN', 'тестирование': 'Тестирование', 
               'nginx': 'Nginx', 'kubernetes': 'Kubernetes', 'vmware': 'VMware', 'redis': 'Redis', 
               'постановка задач разработчикам': 'Постановка задач разработчикам', 'mongodb': 'MongoDB', 
               'scrum': 'SCRUM', 'программирование': 'Программирование', 
               'защиты инфор': 'Cредства информационной защиты', 
               'uml': 'UML', 'разработка по': 'Разработка ПО', 'spring': 'Spring FrameWork', 
               'symfony': 'Symfony', 'cms': 'CMS', 'ansible': 'Ansible', 'django': 'Django FrameWork', 
               'agile': 'AGILE', 'bootstrap': 'BootStrap', 'локаль': 'Администрирование локальных сетей', 'unity': ' Unity', 
                'rabbit' : 'Rabbit', 'vpn' : 'VPN', 'hyper-v' : 'Hyper-v',
                'ajax' : 'AJAX', 'vue' : 'Vue', 'ci/cd' : 'CI/CD',
                'golang' : 'Golang', 'mvc' : 'MVC', 'oracle' : 'Oracle',
                'битрикс' : 'Bitrix', 'bitrix' : 'Bitrix', 'visual studio' : 'Visual Studio',
                'soap' : 'SOAP', 'mac ' : 'Mac OS', 'qt' : 'QT',
                'atlassian confluence' : 'Atlassian Confluence', 'http' : 'HTTP', 'верст' : 'Верстка',
                'сервер' : 'Настройка серверов', 'системный администратор' : 'Системный администратор', 'powershell' : 'PowerShell',
                'yii' : 'Yii', 'access' : 'MS Access', 'kafka' : 'kafka', 
                'delphi': 'Delphi', 'sass': 'SASS', 'solid': 'SOLID', 
               'qa': 'QA', 'jenkins': 'Jenkins', 'devops': 'DevOps', 'webpack': 'Webpack', 
               'backend': 'Backend', 'aws': 'Aws', 'office': 'MS Office', 'asterisk': 'Asterisk', 
               'hibernate': 'Hibernate', 'grafana': 'Grafana', 'mvvm': 'MVVM', 'centos': 'CentOS', 
               'power bi': 'Power BI', 'powerbi': 'Power BI', 'wpf': 'WPF', 'git': 'Git', 
               'test': 'Testing', 'flutter': 'Flutter', 'prometheus': 'Prometheus', 'метрик': 'Метрики', 
               'entity': 'Entity FrameWork', 'gamedev': 'GameDev', 'redmine': 'Redmine', 'osi': 'OSI', 
               'антивирус': 'Антивирусная защита', 'сетевого': 'Настройка сетевого оборудования',
               'frontend': 'Frontend'
              }

def key_skills_transformation(data):
    result = set()
    for skill in data:
        skill = skill.lower()
        for key in skills_dict.keys():
            if key in skill:              
                added = True
                result.add(skills_dict[key])
                skill = skill.replace(key,'')
        if skill == 'go':
            result.add('Golang')
            
    return tuple(result) if result else np.nan

df['key_skills_tr'] = df.key_skills.apply(key_skills_transformation)

In [6]:
df_first = df

In [7]:
df = pd.read_excel('result roles (16.06.2022).xlsx')

In [8]:
# Удаляем неинформативные столбцы.

# df.drop('immediate_redirect_url', axis=1, inplace=True)
df.drop('id', axis=1, inplace=True)
df.drop('premium', axis=1, inplace=True)
df.drop('Unnamed: 0', axis=1, inplace=True)
# df.drop('response_url', axis=1, inplace=True)
df.drop('response_letter_required', axis=1, inplace=True)
df.drop('has_test', axis=1, inplace=True)
# df.drop('address', axis=1, inplace=True)
# df.drop('sort_point_distance', axis=1, inplace=True)
df.drop('url', axis=1, inplace=True)
df.drop('insider_interview', axis=1, inplace=True)
df.drop('alternate_url', axis=1, inplace=True)
df.drop('working_days', axis=1, inplace=True)
df.drop('working_time_intervals', axis=1, inplace=True)
df.drop('working_time_modes', axis=1, inplace=True)
df.drop('accept_temporary', axis=1, inplace=True)
# df.drop('contacts', axis=1, inplace=True)
df.drop('relations', axis=1, inplace=True)
# df.drop('schedule', axis=1, inplace=True)
df.drop('published_at', axis=1, inplace=True)
df.drop('created_at', axis=1, inplace=True)
df.drop('type', axis=1, inplace=True)
# df.drop('department', axis=1, inplace=True)

# df.drop(df[df['archived'] == True].index, axis=0, inplace=True)
df.drop('archived', axis=1, inplace=True)

df.drop('Unnamed: 29', axis=1, inplace=True)
df.drop('Unnamed: 30', axis=1, inplace=True)
df.drop('Unnamed: 31', axis=1, inplace=True)
df.drop('Unnamed: 32', axis=1, inplace=True)
df.drop('Unnamed: 33', axis=1, inplace=True)
df.drop('Unnamed: 34', axis=1, inplace=True)
df.drop('Unnamed: 35', axis=1, inplace=True)
df.drop('Unnamed: 36', axis=1, inplace=True)
df.drop('Unnamed: 37', axis=1, inplace=True)
df.drop('Unnamed: 38', axis=1, inplace=True)
df.drop('Unnamed: 39', axis=1, inplace=True)
df.drop('Unnamed: 40', axis=1, inplace=True)
df.drop('Unnamed: 41', axis=1, inplace=True)
df.drop('Unnamed: 42', axis=1, inplace=True)
df.drop('Unnamed: 43', axis=1, inplace=True)
df.drop('Unnamed: 44', axis=1, inplace=True)
df.drop('Unnamed: 45', axis=1, inplace=True)
df.drop('Unnamed: 46', axis=1, inplace=True)
df.drop('Unnamed: 47', axis=1, inplace=True)
df.drop('Unnamed: 48', axis=1, inplace=True)
df.drop('Unnamed: 49', axis=1, inplace=True)
df.drop('Unnamed: 50', axis=1, inplace=True)

In [9]:
# Преобразуем ячейки сета в удобочитаемый вид.
# Приводим основные профессиональные навыки к одинаковому написанию.

def change_experience(exp):
    try:
        return ast.literal_eval(exp)['name']
    except:
        return np.nan

def change_depart(department):
    try:
        return ast.literal_eval(department)['name']
    except:
        return np.nan
    
def change_employer(string):
    try:
        return ast.literal_eval(string)['name']
    except:
        return np.nan
    
def specializations(data):
    result = []
    try:
        for spec in eval(data):
            result.append((spec["name"],spec["profarea_name"]))
        return tuple(result)
    except:
        return tuple()

def change_skills(skills):
    try:
        return tuple(skill['name'] for skill in ast.literal_eval(skills))
    except:
        return ()

def change_schedule(string):
    try:
        return ast.literal_eval(string)['name']
    except:
        return np.nan
df['department'] = df['department'].apply(change_depart)
df['experience'] = df['experience'].apply(change_experience)
df['specializations'] = df['specializations'].apply(specializations)
df['schedule'] = df['schedule'].apply(change_schedule)
df['employer'] = df['employer'].apply(change_employer)
df['key_skills'] = df['key_skills'].apply(change_skills)
df['key_skills_tr'] = df.key_skills.apply(key_skills_transformation)
df['region'] = df['region'].where(df['region'].isin(df_first['region'].values),np.nan)

df['salary_min'] = df.salary.apply(salary_eval,key='from')
df['salary_max'] = df.salary.apply(salary_eval,key='to')
df['salary_mean'] = df[['salary_min','salary_max']].mean(axis=1)

In [10]:
df_second = df

In [11]:
df_first['month'] = 'Май'
df_second['month'] = 'Июнь'

In [12]:
df = pd.concat([df_first, df_second]).drop_duplicates().reset_index(drop=True)

In [13]:
df = df[['name','salary_mean','experience','schedule','key_skills_tr','specializations','region', 'month']]\
                            .dropna()\
                            .reset_index(drop=True)

In [23]:
def rename(data):
    for name in names:
        if name.lower() in data.lower():
            data = name
    return data

names = pd.Series(df['name'].unique())
names = names[names.str.len() > 14]
df['name'] = df['name'].apply(rename)

In [24]:
df = df.explode('key_skills_tr').explode('specializations')

In [25]:
df['main_spec'] = df['specializations'].apply(lambda x: x[0])
df['sub_spec'] = df['specializations'].apply(lambda x: x[1])
df = df[(df['salary_mean'] < 500000) & (df['salary_mean'] > 9999)].dropna().drop('specializations', axis=1)


In [26]:
df.to_csv(f'VisualV4.csv', header=['name','salary','experience','schedule','skills', 'region', 'month', 'main_spec','sub_spec'])